# P01: Protección ocular frente a radiación láser

Este cuaderno guía el cálculo de la Densidad Óptica (OD) requerida y la selección de gafas (EPO) para tres escenarios típicos, siguiendo los datos de `P1.md`.

Objetivos rápidos:
- Calcular exposición potencial H0 (irradiancia o fluencia) en la córnea en peor caso.
- Obtener OD requerida: OD_req = log10(H0 / HMPE).
- Comparar OD requerida vs. gafas disponibles por longitud de onda.

Escenarios (según práctica):
- Láser 1 (PIV): 532 nm, E = 0.2 J, τ = 8 ns, f = 15 Hz, a = 5 mm, HMPE = 5.0e-7 J/cm².
- Láser 2 (LDA, CW): 514.5 nm, P = 1.5 W, a = 1.2 mm, HMPE = 2.5e-3 W/cm² (0.25 s). Nota: para 10 s, usar 1.0e-3 W/cm².
- Láser 3 (Alineación, CW): 635 nm, P = 4.5 mW, a = 3 mm, HMPE = 2.5e-3 W/cm² (0.25 s).

Gafas (EPO) hipotéticas:
- EPO-001: OD 5+ @ 190–540 nm (VLT 40%)
- EPO-002: OD 2+ @ 630–670 nm; OD 4+ @ 800–1100 nm (VLT 55%)
- EPO-003: OD 7+ @ 190–400 nm; OD 6+ @ 532 nm; OD 5+ @ 1064 nm (VLT 15%)
- EPO-004: OD 2+ @ 10600 nm (CO₂) (VLT 90%)
- EPO-005: OD 7 @ 190–534 nm; OD 6 @ 800–820 nm; OD 5 @ 821–900 nm; OD 6 @ 900–1070 nm (VLT 30%)

Sugerencia de uso:
1) Ejecuta la celda de abajo para ver resultados de los 3 láseres.
2) Ajusta parámetros en la celda de "Escenario personalizado" para explorar variantes.
3) Verifica si la EPO propuesta cubre la λ con OD ≥ OD_req y revisa VLT como criterio secundario.


In [1]:
# Funciones y evaluación de escenarios de la práctica
import math
from typing import List, Dict, Optional, Tuple

# --- Utilidades geométricas ---
def area_cm2_from_diameter_mm(a_mm: float) -> float:
    """Área del haz en cm^2 (A = pi * (d/2)^2), con d en cm."""
    d_cm = a_mm / 10.0
    return math.pi * (d_cm / 2.0) ** 2

# --- Cálculo de H0 ---
def h0_exposure(laser: Dict) -> Tuple[float, str]:
    """Regresa (H0, unidades) según modo (CW -> W/cm^2, pulsado -> J/cm^2)."""
    A = area_cm2_from_diameter_mm(laser["a_mm"])  # cm^2
    if laser["mode"].lower() == "cw":
        H0 = laser["P_W"] / A
        return H0, "W/cm^2"
    else:
        H0 = laser["E_J"] / A
        return H0, "J/cm^2"

# --- OD requerida ---
def od_required(H0: float, HMPE: float) -> float:
    """OD requerida: log10(H0/HMPE). Si H0 < HMPE, retorna 0."""
    if HMPE <= 0:
        raise ValueError("HMPE debe ser positiva.")
    if H0 <= HMPE:
        return 0.0
    return math.log10(H0 / HMPE)

# --- Catálogo de gafas (EPO) ---
# Cada entrada: (min_nm, max_nm, OD)
EPO_CATALOG: Dict[str, Dict] = {
    "EPO-001": {
        "bands": [(190, 540, 5.0)],
        "VLT": 0.40,
    },
    "EPO-002": {
        "bands": [(630, 670, 2.0), (800, 1100, 4.0)],
        "VLT": 0.55,
    },
    "EPO-003": {
        # 532 y 1064 nm como líneas discretas: aproximamos como bandas muy estrechas
        "bands": [(190, 400, 7.0), (531.9, 532.1, 6.0), (1063.9, 1064.1, 5.0)],
        "VLT": 0.15,
    },
    "EPO-004": {
        # CO2 ~ 10600 nm, fuera de nuestro rango visible
        "bands": [(10600, 10600, 2.0)],
        "VLT": 0.90,
    },
    "EPO-005": {
        "bands": [(190, 534, 7.0), (800, 820, 6.0), (821, 900, 5.0), (900, 1070, 6.0)],
        "VLT": 0.30,
    },
}

def epo_max_od_at_lambda(epo_id: str, lam_nm: float) -> Optional[float]:
    bands = EPO_CATALOG[epo_id]["bands"]
    ods = [od for (lo, hi, od) in bands if lo <= lam_nm <= hi]
    return max(ods) if ods else None

# --- Escenarios de la práctica ---
lasers: List[Dict] = [
    {
        "name": "Láser 1 – PIV (pulsado)",
        "lambda_nm": 532.0,
        "mode": "pulsed",
        "a_mm": 5.0,
        "E_J": 0.2,
        "HMPE": 5.0e-7,  # J/cm^2
        "HMPE_units": "J/cm^2",
    },
    {
        "name": "Láser 2 – LDA 514.5 nm (CW)",
        "lambda_nm": 514.5,
        "mode": "CW",
        "a_mm": 1.2,
        "P_W": 1.5,
        "HMPE": 2.5e-3,  # W/cm^2 para 0.25 s
        "HMPE_units": "W/cm^2",
        # Opción alternativa para 10 s
        "HMPE_long": 1.0e-3,  # W/cm^2
    },
    {
        "name": "Láser 3 – Alineación 635 nm (CW)",
        "lambda_nm": 635.0,
        "mode": "CW",
        "a_mm": 3.0,
        "P_W": 4.5e-3,
        "HMPE": 2.5e-3,  # W/cm^2
        "HMPE_units": "W/cm^2",
    },
]

# --- Evaluación ---
def evaluate_laser(laser: Dict) -> Dict:
    A = area_cm2_from_diameter_mm(laser["a_mm"])
    H0, units = h0_exposure(laser)
    od_req = od_required(H0, laser["HMPE"])

    lam = laser["lambda_nm"]
    # Comprobar cada EPO
    candidates = []
    for eid in EPO_CATALOG:
        od_avail = epo_max_od_at_lambda(eid, lam)
        if od_avail is not None:
            safe = od_avail >= od_req
            candidates.append({
                "EPO": eid,
                "OD_avail": od_avail,
                "VLT": EPO_CATALOG[eid]["VLT"],
                "safe": safe,
            })
    # Ordenar: primero los que cumplen, mayor OD y VLT como criterio secundario
    candidates.sort(key=lambda c: (not c["safe"], -c["OD_avail"], -c["VLT"]))

    result = {
        "name": laser["name"],
        "lambda_nm": lam,
        "mode": laser["mode"],
        "a_mm": laser["a_mm"],
        "area_cm2": A,
        "H0": H0,
        "H0_units": units,
        "HMPE": laser["HMPE"],
        "HMPE_units": laser["HMPE_units"],
        "OD_req": od_req,
        "recommendations": candidates,
    }

    # Para LDA, también calcular con HMPE_long si existe
    if "HMPE_long" in laser:
        od_req_long = od_required(H0, laser["HMPE_long"])
        result["OD_req_long"] = od_req_long
        # Re-evaluar seguridad con este criterio más exigente
        for c in result["recommendations"]:
            c["safe_long"] = c["OD_avail"] >= od_req_long
    return result

results = [evaluate_laser(L) for L in lasers]

# --- Presentación simple ---
for r in results:
    print(f"\n=== {r['name']} ===")
    print(f"λ = {r['lambda_nm']} nm | modo = {r['mode']} | a = {r['a_mm']} mm | Área = {r['area_cm2']:.5f} cm²")
    print(f"H0 = {r['H0']:.3e} {r['H0_units']} | HMPE = {r['HMPE']:.2e} {r['HMPE_units']} | OD_req = {r['OD_req']:.2f}")
    if 'OD_req_long' in r:
        print(f"OD_req (10 s) = {r['OD_req_long']:.2f}")
    print("- Gafas evaluadas (mejores primero):")
    for c in r["recommendations"][:5]:
        flags = []
        if c["safe"]: flags.append("OK@0.25s")
        if 'safe_long' in c:
            flags.append("OK@10s" if c["safe_long"] else "FAIL@10s")
        print(f"  {c['EPO']}: OD_avail={c['OD_avail']:.1f}, VLT={int(c['VLT']*100)}%  "
              f"-> {'; '.join(flags) if flags else 'FAIL'}")



=== Láser 1 – PIV (pulsado) ===
λ = 532.0 nm | modo = pulsed | a = 5.0 mm | Área = 0.19635 cm²
H0 = 1.019e+00 J/cm^2 | HMPE = 5.00e-07 J/cm^2 | OD_req = 6.31
- Gafas evaluadas (mejores primero):
  EPO-005: OD_avail=7.0, VLT=30%  -> OK@0.25s
  EPO-003: OD_avail=6.0, VLT=15%  -> FAIL
  EPO-001: OD_avail=5.0, VLT=40%  -> FAIL

=== Láser 2 – LDA 514.5 nm (CW) ===
λ = 514.5 nm | modo = CW | a = 1.2 mm | Área = 0.01131 cm²
H0 = 1.326e+02 W/cm^2 | HMPE = 2.50e-03 W/cm^2 | OD_req = 4.72
OD_req (10 s) = 5.12
- Gafas evaluadas (mejores primero):
  EPO-005: OD_avail=7.0, VLT=30%  -> OK@0.25s; OK@10s
  EPO-001: OD_avail=5.0, VLT=40%  -> OK@0.25s; FAIL@10s

=== Láser 3 – Alineación 635 nm (CW) ===
λ = 635.0 nm | modo = CW | a = 3.0 mm | Área = 0.07069 cm²
H0 = 6.366e-02 W/cm^2 | HMPE = 2.50e-03 W/cm^2 | OD_req = 1.41
- Gafas evaluadas (mejores primero):
  EPO-002: OD_avail=2.0, VLT=55%  -> OK@0.25s


In [2]:
# Escenario personalizado (ajusta valores y vuelve a ejecutar)
import math

custom = {
    "name": "Custom – ejemplo",
    "lambda_nm": 532.0,
    "mode": "pulsed",  # 'CW' o 'pulsed'
    "a_mm": 5.0,
    # Para CW usa P_W; para pulsado usa E_J
    "P_W": 0.5,
    "E_J": 0.05,
    # Define HMPE en las unidades correspondientes al modo
    "HMPE": 5.0e-7,
    "HMPE_units": "J/cm^2",
}

# Reutilizar utilidades definidas
try:
    r = evaluate_laser(custom)
    print(f"\n=== {r['name']} ===")
    print(f"λ = {r['lambda_nm']} nm | modo = {r['mode']} | a = {r['a_mm']} mm | Área = {r['area_cm2']:.5f} cm²")
    print(f"H0 = {r['H0']:.3e} {r['H0_units']} | HMPE = {r['HMPE']:.2e} {r['HMPE_units']} | OD_req = {r['OD_req']:.2f}")
    print("- Gafas evaluadas (mejores primero):")
    for c in r["recommendations"][:5]:
        print(f"  {c['EPO']}: OD_avail={c['OD_avail']:.1f}, VLT={int(c['VLT']*100)}%  -> {'OK' if c['safe'] else 'FAIL'}")
except Exception as e:
    print("Error en escenario personalizado:", e)



=== Custom – ejemplo ===
λ = 532.0 nm | modo = pulsed | a = 5.0 mm | Área = 0.19635 cm²
H0 = 2.546e-01 J/cm^2 | HMPE = 5.00e-07 J/cm^2 | OD_req = 5.71
- Gafas evaluadas (mejores primero):
  EPO-005: OD_avail=7.0, VLT=30%  -> OK
  EPO-003: OD_avail=6.0, VLT=15%  -> OK
  EPO-001: OD_avail=5.0, VLT=40%  -> FAIL
